In [1]:
import pandas as pd
import numpy as np
import math

In [6]:
data_dir = '../Data/cali_example/'
save_dir = '../Data/cali_example_overlap_subsets/'

In [7]:
models = ['AV', 'GS', 'GM']

In [118]:
av_2005 = pd.read_csv(data_dir + 'AV_2005_align.csv')
av_2010 = pd.read_csv(data_dir + 'AV_2010_align.csv')
av_2011 = pd.read_csv(data_dir + 'AV_2011_align.csv')
gm = pd.read_csv(data_dir + 'gb_gm_ca_mapped_2010_2016/gb_gm_ca_mapped_2010_2016/GM_align_2010_2016.csv')
gb = pd.read_csv(data_dir + 'gb_gm_ca_mapped_2010_2016/gb_gm_ca_mapped_2010_2016/GB_align_2010_2016.csv')

In [120]:
av_2005.columns = ['lon', 'lat', 'pm25']
av_2010.columns = ['lon', 'lat', 'pm25'] 
av_2011.columns = ['lon', 'lat', 'pm25']

In [122]:
time_2005 = np.repeat(2005, av_2005.shape[0])
time_2010 = np.repeat(2010, av_2010.shape[0])
time_2011 = np.repeat(2011, av_2011.shape[0])

In [123]:
av_2005['time'] = time_2005
av_2010['time'] = time_2010
av_2011['time'] = time_2011

In [126]:
av_2005_2010 = av_2005.append(av_2010)
av_2005_2011 = av_2005_2010.append(av_2011)

In [127]:
gm_2005_2011 = gm.loc[gm.time.isin([2005, 2010, 2011])]
gb_2005_2011 = gb.loc[gb.time.isin([2005, 2010, 2011])]
# gm_2005 = gm.loc[gm.time.isin([2005])]
# gb_2005 = gb.loc[gb.time.isin([2005])]

In [129]:
intersecting_lat_lon = pd.merge(av_2005_2011, gm_2005_2011, how='inner', on=['lon', 'lat', 'time'])

In [232]:
intersecting_lat_lon.head()

,lon,lat,pm25_av,time,Unnamed: 0,pm25_gm
0,-124.405,42.005,NaN,2005,983665,2.048217
1,-124.395,42.005,NaN,2005,983666,2.048217
2,-124.385,42.005,NaN,2005,983667,2.048217
3,-124.375,42.005,NaN,2005,983668,2.048217
4,-124.365,42.005,NaN,2005,983669,2.048217


In [233]:
intersecting_lat_lon.columns = ['lon', 'lat', 'pm25_av', 'time', 'Unnamed: 0', 'pm25_gm']

In [234]:
intersecting_lat_lon_all = pd.merge(gb_2005_2011, intersecting_lat_lon, how = 'inner', on=['lon', 'lat', 'time'])

In [235]:
intersecting_lat_lon_all.head()

,lon,lat,pm25,time,pm25_av,Unnamed: 0,pm25_gm
0,-114.195,32.595,8.119813,2005,7.1,13209162,3.295776
1,-114.185,32.595,8.119813,2005,7.0,13209163,3.295776
2,-114.175,32.595,8.119813,2005,7.1,13209164,3.295776
3,-114.165,32.595,8.119813,2005,7.1,13209165,3.295776
4,-114.155,32.595,8.119813,2005,7.1,13209166,3.295776


In [236]:
intersecting_lat_lon_all.columns = ['Unnamed: 0_x', 'time', 'y', 'x', 'pred_gb', 'pred_av', 'Unnamed: 0_y',
       'pred_gm']

In [237]:
intersecting_lat_lon_all = intersecting_lat_lon_all[['time','x', 'y' 'pred_gb', 'pred_av', 'pred_gm']]

In [224]:
all_preds_file = save_dir + 'predictions_2005_2011.csv'

In [227]:
intersecting_lat_lon_all.to_csv(all_preds_file, index = False)

In [142]:
intersecting_lat_lon_all = intersecting_lat_lon_all[~intersecting_lat_lon_all['pm25_av'].isnull()]

In [144]:
intersecting_lat_lon_all.head()

,Unnamed: 0_x,time,lat,lon,pm25,pm25_av,Unnamed: 0_y,pm25_gm
0,246,2005,32.595,-114.195,8.119813,7.1,13209162,3.295776
1,247,2005,32.595,-114.185,8.119813,7.0,13209163,3.295776
2,248,2005,32.595,-114.175,8.119813,7.1,13209164,3.295776
3,249,2005,32.595,-114.165,8.119813,7.1,13209165,3.295776
4,250,2005,32.595,-114.155,8.119813,7.1,13209166,3.295776


In [163]:
av_2005_2011 = intersecting_lat_lon_all[['lon', 'lat', 'pm25_av', 'time']]
gm_2005_2011 = intersecting_lat_lon_all[['lon', 'lat', 'pm25_gm', 'time']]
gb_2005_2011 = intersecting_lat_lon_all[['lon', 'lat', 'pm25', 'time']]

In [210]:
av_2005_2011_file = data_dir + 'AV_2005_2011_align.csv'
gm_2005_2011_file = data_dir + 'GM_2005_2011_align.csv'
gb_2005_2011_file = data_dir + 'GB_2005_2011_align.csv'
av_2005_2011.to_csv(av_2005_2011_file, index = False)
gm_2005_2011.to_csv(gm_2005_2011_file, index = False)
gb_2005_2011.to_csv(gb_2005_2011_file, index = False)

In [165]:
av_2005_2011.head()

,lon,lat,pm25_av,time
0,-114.195,32.595,7.1,2005
1,-114.185,32.595,7.0,2005
2,-114.175,32.595,7.1,2005
3,-114.165,32.595,7.1,2005
4,-114.155,32.595,7.1,2005


In [166]:
av_2005_2011.columns = ['lon', 'lat', 'pm25', 'time']
gm_2005_2011.columns = ['lon', 'lat', 'pm25', 'time']

In [149]:
min_lon = min(av_2005_2011.lon)
max_lon = max(av_2005_2011.lon)
min_lat = min(av_2005_2011.lat)
max_lat = max(av_2005_2011.lat)

upper_left = (max_lat, min_lon)
upper_right = (max_lat, max_lon)
lower_left = (min_lat, min_lon)
lower_right = (min_lat, max_lon)

In [104]:
print (upper_left, upper_right, lower_left, lower_right)

(42.005, -124.395) (42.005, -114.135) (32.535, -124.395) (32.535, -114.135)


In [105]:
test = av_2005.loc[(av_2005.lon >= min_lon) & (av_2005.lon <= max_lon) & (av_2005.lat >= min_lat) & (av_2005.lat <= max_lat)]
test.shape

(723716, 3)

In [151]:
X_valid = np.asarray(av_2005_2011[["lon", "lat"]].values.tolist()).astype(np.float32)

In [152]:
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

In [153]:
X_scale

array([10.2599945,  9.470001 ], dtype=float32)

In [154]:
X_valid[:5]

array([[-114.195,   32.595],
       [-114.185,   32.595],
       [-114.175,   32.595],
       [-114.165,   32.595],
       [-114.155,   32.595]], dtype=float32)

In [155]:
def round_nearest(x, a):
    return round(round(x / a) * a, -int(math.floor(math.log10(a))))

In [156]:
split_range = [X_scale[0]/6, X_scale[1]/6]
split_range = [ round(elem, 4) for elem in split_range ]
split_lon_range = split_range[0]
split_lat_range = split_range[1]
print (split_lon_range, split_lat_range)

1.71 1.5783


In [188]:
start_split_lon = []
for i in range(10):
    start_split_lon.append(min_lon + i*split_lon_range)
    
start_split_lat = []
for i in range(10):
    start_split_lat.append(min_lat + i*split_lat_range)

In [189]:
a = [round_nearest(x, 0.005) for x in start_split_lat]
b = [x + 0.005 if (str(x)[-1] != '5' or len(str(x).split('.')[1]) !=  3) else x for x in a]
start_split_lat = [round(x, -int(math.floor(math.log10(0.005)))) for x in b]

In [190]:
start_split_lat, start_split_lon

([32.535,
  34.115,
  35.695,
  37.275,
  38.855,
  40.425,
  42.005,
  43.585,
  45.165,
  46.745],
 [-124.395,
  -122.685,
  -120.975,
  -119.265,
  -117.55499999999999,
  -115.845,
  -114.13499999999999,
  -112.425,
  -110.715,
  -109.005])

In [191]:
start_split_lon = [round(x, 3) for x in start_split_lon]
end_split_lon = start_split_lon[1:]
end_split_lon.append(max_lon)
start_split_lon = [x - 0.5 for x in start_split_lon[1:]]
start_split_lon.insert(0, min_lon)
print (start_split_lon)
print (end_split_lon)

end_split_lat = start_split_lat[1:]
end_split_lat.append(max_lat)
start_split_lat = [x - 0.5 for x in start_split_lat[1:]]
start_split_lat.insert(0, min_lat)

lon_range = list(zip(start_split_lon, end_split_lon))
lat_range = list(zip(start_split_lat, end_split_lat))

[-124.395, -123.185, -121.475, -119.765, -118.055, -116.345, -114.635, -112.925, -111.215, -109.505]
[-122.685, -120.975, -119.265, -117.555, -115.845, -114.135, -112.425, -110.715, -109.005, -114.135]


In [192]:
lat_range

[(32.535, 34.115),
 (33.615, 35.695),
 (35.195, 37.275),
 (36.775, 38.855),
 (38.355, 40.425),
 (39.925, 42.005),
 (41.505, 43.585),
 (43.085, 45.165),
 (44.665, 46.745),
 (46.245, 42.005)]

In [193]:
lon_range

[(-124.395, -122.685),
 (-123.185, -120.975),
 (-121.475, -119.265),
 (-119.765, -117.555),
 (-118.055, -115.845),
 (-116.345, -114.135),
 (-114.635, -112.425),
 (-112.925, -110.715),
 (-111.215, -109.005),
 (-109.505, -114.135)]

In [204]:
monthly_pred = pd.DataFrame(columns=['lon', 'lat', 'pm25', 'time'])
subset_num = 1
num_subset = {'AV' : 1, 'GB' : 1, 'GM' : 1}
num_coords = 0
models = [av_2005_2011, gm_2005_2011, gb_2005_2011]
model_list = ['AV', 'GM', 'GB']

for m, model in enumerate(models):
    for i in range(len(lat_range)):
        for j in range(len(lon_range)):
            df_subset = model.loc[(model.lon >= lon_range[j][0]) & (model.lon <= lon_range[j][1]) & (model.lat >= lat_range[i][0]) & (model.lat <= lat_range[i][1])]
            df_subset = df_subset[['lon', 'lat', 'pm25', 'time']]
            monthly_pred = monthly_pred.append(df_subset)
            if (df_subset.shape[0] != 0):
                new_file = '{}{}_2005_2011_align.{}.csv'.format(save_dir, model_list[m], num_subset[model_list[m]])
#                 print (new_file)
                num_subset[model_list[m]] += 1
#                 df_subset.to_csv(new_file, index = False)

../Data/cali_example_overlap_subsets/GB_2005_2011_align.1.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.2.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.3.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.4.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.5.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.6.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.7.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.8.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.9.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.10.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.11.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.12.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.13.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.14.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.15.csv
../Data/cali_example_overlap_subsets/GB_2005_2011_align.16.csv
.

In [205]:
print (num_coords/3)

0.0


In [206]:
monthly_pred.drop_duplicates(inplace = True)

In [157]:
av_jan.shape

(723716, 5)

In [24]:
split_range

[1.71, 1.578]

In [ ]:
split_poitns = []

In [9]:
#this creates different sized splits

num_lon_splits = 6
total_coords = 0
lon_splits = np.array_split(np.unique(np.sort(av_jan.lon.values)), num_lon_splits)
num_subset = {'AV' : 1, 'GS' : 1, 'GM' : 1}

for model in models:
    model_df = pd.read_csv('{}{}_clean_20101_align.csv'.format(data_dir,model))
    for i in range(num_lon_splits):
        sub_lon = lon_splits[i]
        df_sub = model_df.loc[model_df.lon.isin(sub_lon)]
        num_lat_splits = 5
        lat_splits = np.array_split(np.unique(np.sort(df_sub.lat.values)), num_lat_splits)
        for j in range(num_lat_splits):
            sub_lat = lat_splits[j]
            df_sub_sub = df_sub.loc[df_sub.lat.isin(sub_lat)]
            total_coords += df_sub_sub.shape[0]
            new_file = '{}{}_clean_20101_align.{}.csv'.format(data_dir, model, num_subset[model])
            print (new_file)
            num_subset[model] += 1
            print (df_sub_sub.shape)
#             df_sub_sub.to_csv(new_file, index = False)

../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.1.csv
(2879, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.2.csv
(8324, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.3.csv
(10757, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.4.csv
(12501, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.5.csv
(11819, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.6.csv
(8040, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.7.csv
(16928, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.8.csv
(22216, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.9.csv
(22230, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.10.csv
(22230, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.11.csv
(16685, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.12.csv
(27531, 5)
../Data/cali_pred_data_2010_monthly/AV_clean_20101_align.13.csv
(27531, 5)
../Data/cali_pred_data_2010_monthly/A

In [83]:
# using coordinates from R
num_coords = 0
num_r_subsegs = 36
num_subsegs = {'AV' : 0, 'GS' : 0, 'GM' : 0}

for model in models:
    model_df = pd.read_csv('{}{}_clean_20101_align.csv'.format(data_dir,model))
    for i in range(1, num_r_subsegs + 1):
        file_name = '{}coordinates_{}.csv'.format(data_dir, i)
        coordinates = pd.read_csv(file_name)
        lon_list = list(np.around(np.array(np.unique(coordinates.x.values)),3))
        lat_list = list(np.around(np.array(np.unique(coordinates.y.values)),3))
        df_sub = model_df.loc[(model_df.lon.isin(lon_list)) & (model_df.lat.isin(lat_list))]
        if (df_sub.shape[0] != 0):
            df_sub = df_sub[['lon', 'lat', 'pm25']]
            num_subsegs[model] += 1
            new_file = '{}{}_clean_20101_align.{}.csv'.format(data_dir, model, num_subsegs[model])
            df_sub.to_csv(new_file, index = False)

# Nationwide

In [2]:
data_dir = '../Data/nationwide/'
save_dir = '../Data/nationwide_subsets/'

In [3]:
av_2010 = pd.read_csv(data_dir + 'AV_2010_align.csv')

In [6]:
gbd_2010 = pd.read_csv(data_dir + 'GBD_2010_align.csv')

In [7]:
gbd_2010.shape

(8348297, 5)

In [4]:
av_2010.shape

(8497549, 3)

In [72]:
intersecting_av_gbd = pd.merge(av_2010, gbd_2010, how = 'left', on=['lon', 'lat'])

In [76]:
av_2010.shape[0]- 149252 

8348297

In [74]:
intersecting_av_gbd.isna().sum()

lon                0
lat                0
pm25_x             0
Unnamed: 0    149252
time          149252
pm25_y        149252
dtype: int64

In [77]:
non_intersecting = intersecting_av_gbd.loc[intersecting_av_gbd.time.isna()]

In [87]:
non_intersecting.loc[non_intersecting.pm25_x != -999.99].head(200)

,lon,lat,pm25_x,Unnamed: 0,time,pm25_y
300,-124.745,48.165,4.0,NaN,NaN,NaN
301,-124.745,48.175,4.0,NaN,NaN,NaN
323,-124.745,48.395,4.1,NaN,NaN,NaN
359,-124.735,48.145,3.9,NaN,NaN,NaN
361,-124.735,48.165,4.0,NaN,NaN,NaN
362,-124.735,48.175,4.0,NaN,NaN,NaN
384,-124.735,48.395,4.1,NaN,NaN,NaN
420,-124.725,48.145,3.9,NaN,NaN,NaN
422,-124.725,48.165,3.9,NaN,NaN,NaN
423,-124.725,48.175,3.9,NaN,NaN,NaN


In [3]:
av_2004 = pd.read_csv(data_dir + 'AV_2004_align.csv')

In [4]:
min_lon = min(av_2004.lon)
max_lon = max(av_2004.lon)
min_lat = min(av_2004.lat)
max_lat = max(av_2004.lat)

In [5]:
X_valid = np.asarray(av_2004[["lon", "lat"]].values.tolist()).astype(np.float32)
X_scale = np.max(X_valid, axis=0) - np.min(X_valid, axis=0)

In [6]:
def round_nearest(x, a):
    return round(round(x / a) * a, -int(math.floor(math.log10(a))))

In [7]:
def round_location(locations):
    round_loc = [round_nearest(x, 0.005) for x in locations]
    round_loc = [x + 0.005 if (str(x)[-1] != '5' or len(str(x).split('.')[1]) !=  3) else x for x in round_loc]
    round_loc =  [round(x, -int(math.floor(math.log10(0.005)))) for x in round_loc]
    return round_loc

In [8]:
def get_splits(X_val, num_splits):
    split_range = [X_scale[0]/num_splits, X_scale[1]/num_splits]
    split_range = [round(elem, 4) for elem in split_range ]
    split_lon_range = split_range[0]
    split_lat_range = split_range[1]

    start_split_lon = []
    for i in range(num_splits):
        start_split_lon.append(min_lon + i*split_lon_range)

    start_split_lat = []
    for i in range(num_splits):
        start_split_lat.append(min_lat + i*split_lat_range)

    start_split_lon = round_location(start_split_lon)
    start_split_lat = round_location(start_split_lat)
    
    end_split_lon = start_split_lon[1:]
    end_split_lon.append(max_lon)
    start_split_lon = [x - 0.5 for x in start_split_lon[1:]]
    start_split_lon.insert(0, min_lon)

    end_split_lat = start_split_lat[1:]
    end_split_lat.append(max_lat)
    start_split_lat = [x - 0.5 for x in start_split_lat[1:]]
    start_split_lat.insert(0, min_lat)
    
    lon_range = list(zip(start_split_lon, end_split_lon))
    lat_range = list(zip(start_split_lat, end_split_lat))
    
    return lon_range, lat_range

In [9]:
lon_range, lat_range = get_splits(X_valid, 49)

In [10]:
av_2010_2016 = pd.DataFrame(columns=['lon', 'lat', 'pm25', 'time'])
for i in range(2010, 2017):
    av_file_name = '{}AV_{}_align.csv'.format(data_dir, i)
    av_i = pd.read_csv(av_file_name)
    time_i = np.repeat(i, av_i.shape[0])
    av_i['time'] = time_i
    av_2010_2016 = av_2010_2016.append(av_i)

av_2010_2016 = av_2010_2016.loc[av_2010_2016.pm25 != -999.99]

In [12]:
av_2010_2016.shape[0]/7

8273037.0

In [19]:
av_2004.shape

(8497549, 3)

In [20]:
av_2004.shape

(8497549, 3)

In [25]:
av_no_null = av_2004.loc[av_2004.pm25 != -999.99]

In [35]:
full_data = pd.DataFrame(columns=['lon', 'lat', 'pm25'])
model = av_2010_2016
subset_num = 0
subset_shapes = []

most_recent_subset = pd.DataFrame(columns=['lon', 'lat', 'pm25'])

for i in range(len(lat_range)):
    for j in range(len(lon_range)):
        df_subset = model.loc[(model.lon >= lon_range[j][0]) & (model.lon <= lon_range[j][1]) & (model.lat >= lat_range[i][0]) & (model.lat <= lat_range[i][1])]
        df_subset = df_subset[['lon', 'lat', 'pm25']]
        if (df_subset.shape[0] != 0):
            if (df_subset.shape[0] > 700):
                most_recent_subset = df_subset
                full_data = full_data.append(df_subset)
                subset_shapes.append(most_recent_subset.shape[0])
                subset_num += 1
                new_file = '{}AV_2010_2016_align.{}.csv'.format(save_dir, subset_num)
                most_recent_subset.to_csv(new_file, index = False)
            else:
                most_recent_subset = most_recent_subset.append(df_subset)
                subset_shapes.pop()
                subset_shapes.append(most_recent_subset.shape[0])
                most_recent_subset.to_csv(new_file, index = False)  

DF subset: (597, 3)
Most recent subset: (597, 3)
1
DF subset: (505, 3)
Most recent subset: (505, 3)
2
DF subset: (843, 3)
Most recent subset: (843, 3)
3
DF subset: (3159, 3)
Most recent subset: (3159, 3)
4
DF subset: (284, 3)
Most recent subset: (284, 3)
5
DF subset: (139, 3)
../Data/nationwide_subsets/AV_2010_2016_align.5.csv
Most recent subset: (423, 3)
5
DF subset: (1730, 3)
Most recent subset: (1730, 3)
6
DF subset: (8526, 3)
Most recent subset: (8526, 3)
7
DF subset: (1303, 3)
Most recent subset: (1303, 3)
8


KeyboardInterrupt: 

In [31]:
print (len(subset_shapes))
print (np.min(subset_shapes))

1556
276


In [32]:
full_data.drop_duplicates(inplace = True)
# print (av_2010_2016.shape[0]/7)
print (full_data.shape)

(8273037, 3)


# 1605 subsets or 1590?